<a href="https://colab.research.google.com/github/devansh20/intern/blob/main/StaticQuantizeDistilbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#https://www.philschmid.de/static-quantization-optimum

In [1]:
!pip install "optimum[onnxruntime]==1.2.2" evaluate[evaluator] sklearn mkl-include mkl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 88 kB 4.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 68 kB 7.8 MB/s 
     |████████████████████████████████| 1.2 MB 43.9 MB/s 
     |████████████████████████████████| 46 kB 4.4 MB/s 
     |████████████████████████████████| 4.2 MB 54.1 MB/s 
     |████████████████████████████████| 86 kB 5.6 MB/s 
     |████████████████████████████████| 1.0 MB 53.0 MB/s 
     |████████████████████████████████| 12.8 MB 48.0 MB/s 
     |████████████████████████████████| 5.2 MB 58.2 MB/s 
     |████████████████████████████████| 346 kB 65.6 MB/s 
     |████████████████████████████████| 212 kB 69.6 MB/s 
     |████████████████████████████████| 86 kB 7.3 MB/s 
     |████████████████████████████████| 140 kB 71.3 MB/s 
     |████████████████████████

In [2]:
from optimum.onnxruntime import ORTModelForSequenceClassification
from transformers import AutoTokenizer
from pathlib import Path
model_id="optimum/distilbert-base-uncased-finetuned-banking77"
dataset_id="banking77"
onnx_path = Path("onnx")
 
# load vanilla transformers and convert to onnx
model = ORTModelForSequenceClassification.from_pretrained(model_id, from_transformers=True)
tokenizer = AutoTokenizer.from_pretrained(model_id)
 
# save onnx checkpoint and tokenizer
model.save_pretrained(onnx_path)
tokenizer.save_pretrained(onnx_path)

Downloading:   0%|          | 0.00/5.81k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/333 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/695k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.67k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

('onnx/tokenizer_config.json',
 'onnx/special_tokens_map.json',
 'onnx/vocab.txt',
 'onnx/added_tokens.json',
 'onnx/tokenizer.json')

In [3]:
from optimum.onnxruntime import ORTQuantizer
from optimum.onnxruntime.configuration import AutoQuantizationConfig
from onnxruntime.quantization import QuantFormat, QuantizationMode
quantizer = ORTQuantizer.from_pretrained(model_id, feature=model.pipeline_task)
qconfig = AutoQuantizationConfig.avx512_vnni(
     is_static=True,
     format=QuantFormat.QOperator,
     mode=QuantizationMode.QLinearOps,
     per_channel=True,
     operators_to_quantize=["MatMul", "Add" ]
 )

In [11]:
import os
from functools import partial
from optimum.onnxruntime.configuration import AutoCalibrationConfig
 
def preprocess_fn(ex, tokenizer):
     return tokenizer(ex["text"],padding="longest")

calibration_samples = 256
calibration_dataset = quantizer.get_calibration_dataset(
     dataset_id,
     preprocess_function=partial(preprocess_fn, tokenizer=quantizer.tokenizer),
     num_samples=calibration_samples,
     dataset_split="train",
 )
calibration_config = AutoCalibrationConfig.percentiles(calibration_dataset, percentile=99.99239080907178)

shards=16
for i in range(shards):
     shard = calibration_dataset.shard(shards, i)
     quantizer.partial_fit(
     dataset=shard,
     calibration_config=calibration_config,
     onnx_model_path=onnx_path / "model.onnx",
     operators_to_quantize=qconfig.operators_to_quantize,
     batch_size=calibration_samples//shards,
     use_external_data_format=False,
     )



ranges = quantizer.compute_ranges()
os.remove("augmented_model.onnx")

Using custom data configuration default
Reusing dataset banking77 (/root/.cache/huggingface/datasets/banking77/default/1.1.0/aec0289529599d4572d76ab00c8944cb84f88410ad0c9e7da26189d31f62a55b)
Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/banking77/default/1.1.0/aec0289529599d4572d76ab00c8944cb84f88410ad0c9e7da26189d31f62a55b/cache-e1ec9724da02bf72.arrow


  0%|          | 0/1 [00:00<?, ?ba/s]

Finding optimal threshold for each tensor using percentile algorithm ...
Number of tensors : 218
Number of histogram bins : 2048
Percentile : (0.00760919092822121,99.99239080907178)


In [21]:
#from utils import create_quantization_preprocessor

#quantization_preprocessor = create_quantization_preprocessor()

quantizer.export(
     onnx_model_path=onnx_path / "model.onnx",
     onnx_quantized_model_output_path=onnx_path / "model-quantized.onnx",
     calibration_tensors_range=ranges,
     quantization_config=qconfig,
     #preprocessor=quantization_preprocessor,
 )

PosixPath('onnx/model-quantized.onnx')

In [17]:
!pip install python-utils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
import os
size = os.path.getsize(onnx_path / "model.onnx")/(1024*1024)
quantized_model = os.path.getsize(onnx_path / "model-quantized.onnx")/(1024*1024) 
print(f"Model file size: {size:.2f} MB")
print(f"Quantized Model file size: {quantized_model:.2f} MB")

Model file size: 255.68 MB
Quantized Model file size: 134.32 MB


In [23]:
from optimum.onnxruntime import ORTModelForSequenceClassification
from transformers import pipeline, AutoTokenizer

model = ORTModelForSequenceClassification.from_pretrained(onnx_path,file_name="model-quantized.onnx")
tokenizer = AutoTokenizer.from_pretrained(onnx_path)

clx = pipeline("text-classification",model=model, tokenizer=tokenizer)
 
clx("What is the exchange rate like on this app?")

[{'label': 'declined_cash_withdrawal', 'score': 0.08346515148878098}]

In [24]:
from evaluate import evaluator
from datasets import load_dataset 
 
eval = evaluator("text-classification")
eval_dataset = load_dataset("banking77", split="test")
 
results = eval.compute(
     model_or_pipeline=clx,
     data=eval_dataset,
     metric="accuracy",
     input_column="text",
     label_column="label",
     label_mapping=model.config.label2id,
     strategy="simple",
)
print(results) 
print(f"Vanilla model: 92.5%")
print(f"Quantized model: {results['accuracy']*100:.2f}%")
print(f"The quantized model achieves {round(results['accuracy']/0.925,4)*100:.2f}% accuracy of the fp32 model")

Using custom data configuration default
Reusing dataset banking77 (/root/.cache/huggingface/datasets/banking77/default/1.1.0/aec0289529599d4572d76ab00c8944cb84f88410ad0c9e7da26189d31f62a55b)


{'accuracy': 0.012662337662337663}
Vanilla model: 92.5%
Quantized model: 1.27%
The quantized model achieves 1.37% accuracy of the fp32 model
